In [20]:
# --------------------------------------------
# IMPORT LIBRAIRIES
# --------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib

import func_causalite as f
importlib.reload(f)

pd.set_option('display.max_rows', None)

# Simulation 1 : 2 covariables sans termes croisés

In [ ]:
Nobs = 1000 # nombre d'observations
alpha_tau = 1 #effet du traitement
alpha_eZ = [2,2,0,0] #effet des covariables sur le propensity score
NX = np.size(alpha_eZ) # nombre de covariables
alpha_eY = [-2,-2,0,0] #effet des covariables sur la variable d'intérêt
alphaXCrossX_eZ = np.zeros((NX,NX)) #effet croisé des covariables sur le propensity score
alphaXCrossX_eY = np.zeros((NX,NX)) #effet croisé des covariables sur la variable d'intérêt
alphaXCrossZ_eY = [0,0,0,0]
df, tau_pf, tau_causal = f.genData(Nobs,alpha_tau,alpha_eZ,alpha_eY,alphaXCrossX_eZ,alphaXCrossX_eY,alphaXCrossZ_eY)
print('VALEURS TAU SIMULATION :')
print('tau_pf = E[Y,Z=1] - E[Y,Z=0] =', round(tau_pf,ndigits=3))   
print('tau_causal = E[Y(1)] - E[Y(0)] =', round(tau_causal ,ndigits=3))
print('')

f.print_moy_strat(df)
print('')

f.print_res_sim(df)

VALEURS TAU SIMULATION :
tau_pf = E[Y,Z=1] - E[Y,Z=0] = -0.127
tau_causal = E[Y(1)] - E[Y(0)] = 0.175

RESULTATS DE LA MOYENNE STRATIFIÉE :
             E[Y|Z=0]  E[Y|Z=1]  N(Z=0)  N(Z=1)      Diff  Poids
X1 X2 X3 X4                                                     
0  0  0  0   0.750000  0.950000   168.0    40.0  0.200000  208.0
         1   0.666667  0.736842    78.0    19.0  0.070175   97.0
      1  0   0.707692  0.772727    65.0    22.0  0.065035   87.0
         1   0.735294  0.909091    34.0    11.0  0.173797   45.0
   1  0  0   0.166667  0.428571    36.0    56.0  0.261905   92.0
         1   0.352941  0.466667    17.0    30.0  0.113725   47.0
      1  0   0.500000  0.500000    18.0    40.0  0.000000   58.0
         1   0.400000  0.750000     5.0    20.0  0.350000   25.0
1  0  0  0   0.216216  0.465517    37.0    58.0  0.249301   95.0
         1   0.333333  0.517241    15.0    29.0  0.183908   44.0
      1  0   0.294118  0.375000    17.0    32.0  0.080882   49.0
         1   0.

In [ ]:
lbLogY,ubLogY,lbArbY,ubArbY,lbLogE,ubLogE,lbArbE,ubArbE = f.calculBootstrap(df,100)
print('')
print('INTERVALLE DE CONFIANCE IPW reg. log.')
print(str(lbLogE) + '-' + str(ubLogE))
print('INTERVALLE DE CONFIANCE IPW arb. dec.')
print(str(lbArbE) + '-' + str(ubArbE))
print('INTERVALLE DE CONFIANCE OR reg. log.')
print(str(lbLogY) + '-' + str(ubLogY))
print('INTERVALLE DE CONFIANCE OR arb. dec.')
print(str(lbArbY) + '-' + str(ubArbY))

# Simulation 2 : 2 covariables avec termes croisés pour Y

In [26]:
Nobs = 1000000 # nombre d'observations
alpha_tau = 1 #effet du traitement
alpha_eZ = [2,2,0,0] #effet des covariables sur le propensity score
NX = np.size(alpha_eZ) # nombre de covariables
alpha_eY = [-2,-2,0,0] #effet des covariables sur la variable d'intérêt
alphaXCrossX_eZ = np.zeros((NX,NX)) #effet croisé des covariables sur le propensity score
alphaXCrossX_eY = np.zeros((NX,NX)) #effet croisé des covariables sur la variable d'intérêt
alphaXCrossX_eZ[0,1] = 0 #effet croisé des covariables sur le propensity score
alphaXCrossX_eY[0,1] = -2 #effet croisé des covariables sur la variable d'intérêt
alphaXCrossZ_eY = [0,0,0,0]
df, tau_pf, tau_causal = f.genData(Nobs,alpha_tau,alpha_eZ,alpha_eY,alphaXCrossX_eZ,alphaXCrossX_eY,alphaXCrossZ_eY)
print('VALEURS TAU SIMULATION :')
print('tau_pf = E[Y,Z=1] - E[Y,Z=0] =', round(tau_pf,ndigits=3))   
print('tau_causal = E[Y(1)] - E[Y(0)] =', round(tau_causal ,ndigits=3))
print('')

f.print_moy_strat(df)
print('')

f.print_res_sim(df)

VALEURS TAU SIMULATION :
tau_pf = E[Y,Z=1] - E[Y,Z=0] = -0.117
tau_causal = E[Y(1)] - E[Y(0)] = 0.166

RESULTATS DE LA MOYENNE STRATIFIÉE :
             E[Y|Z=0]  E[Y|Z=1]    N(Z=0)   N(Z=1)      Diff     Poids
X1 X2 X3 X4                                                           
0  0  0  0   0.772397  0.903402  156927.0  41326.0  0.131005  198253.0
         1   0.773101  0.901006   78383.0  20779.0  0.127905   99162.0
      1  0   0.772268  0.899247   78285.0  20724.0  0.126979   99009.0
         1   0.771651  0.905538   38866.0  10311.0  0.133886   49177.0
   1  0  0   0.315431  0.557934   33367.0  64988.0  0.242502   98355.0
         1   0.317189  0.558781   16662.0  32757.0  0.241593   49419.0
      1  0   0.307201  0.556727   16761.0  32524.0  0.249526   49285.0
         1   0.312071  0.559809    8450.0  16377.0  0.247738   24827.0
1  0  0  0   0.314040  0.556514   33069.0  65055.0  0.242473   98124.0
         1   0.313408  0.551663   16729.0  32509.0  0.238255   49238.0
      1 

In [ ]:
lbLogY,ubLogY,lbArbY,ubArbY,lbLogE,ubLogE,lbArbE,ubArbE = f.calculBootstrap(df,100)
print('')
print('INTERVALLE DE CONFIANCE IPW reg. log.')
print(str(lbLogE) + '-' + str(ubLogE))
print('INTERVALLE DE CONFIANCE IPW arb. dec.')
print(str(lbArbE) + '-' + str(ubArbE))
print('INTERVALLE DE CONFIANCE OR reg. log.')
print(str(lbLogY) + '-' + str(ubLogY))
print('INTERVALLE DE CONFIANCE OR arb. dec.')
print(str(lbArbY) + '-' + str(ubArbY))

# Simulation 3 : 2 covariables avec termes croisés pour Z

In [547]:
Nobs = 1000000 # nombre d'observations
alpha_tau = 1 #effet du traitement
alpha_eZ = [2,2,0,0] #effet des covariables sur le propensity score
NX = np.size(alpha_eZ) # nombre de covariables
alpha_eY = [-2,-2,0,0] #effet des covariables sur la variable d'intérêt
alphaXCrossX_eZ = np.zeros((NX,NX)) #effet croisé des covariables sur le propensity score
alphaXCrossX_eY = np.zeros((NX,NX)) #effet croisé des covariables sur la variable d'intérêt
alphaXCrossX_eZ[0,1] = -2 #effet croisé des covariables sur le propensity score
alphaXCrossX_eY[0,1] = 0 #effet croisé des covariables sur la variable d'intérêt
alphaXCrossZ_eY = [0,0,0,0]
df, tau_pf, tau_causal = f.genData(Nobs,alpha_tau,alpha_eZ,alpha_eY,alphaXCrossX_eZ,alphaXCrossX_eY,alphaXCrossZ_eY)
print('VALEURS TAU SIMULATION :')
print('tau_pf = E[Y,Z=1] - E[Y,Z=0] =', round(tau_pf,ndigits=3))   
print('tau_causal = E[Y(1)] - E[Y(0)] =', round(tau_causal ,ndigits=3))
print('')

f.print_moy_strat(df)
print('')

f.print_res_sim(df)

VALEURS TAU SIMULATION :
tau_pf = E[Y,Z=1] - E[Y,Z=0] = -0.044
tau_causal = E[Y(1)] - E[Y(0)] = 0.177

RESULTATS DE LA MOYENNE STRATIFIÉE :
             E[Y|Z=0]  E[Y|Z=1]    N(Z=0)   N(Z=1)      Diff     Poids
X1 X2 X3 X4                                                           
0  0  0  0   0.733709  0.878182  148518.0  48901.0  0.144473  197419.0
         1   0.730766  0.882801   74255.0  24309.0  0.152035   98564.0
      1  0   0.731953  0.882391   74834.0  24522.0  0.150438   99356.0
         1   0.732603  0.880045   37405.0  12338.0  0.147443   49743.0
   1  0  0   0.272054  0.504038   28884.0  70213.0  0.231984   99097.0
         1   0.272068  0.502436   14195.0  34892.0  0.230368   49087.0
      1  0   0.268267  0.495925   14452.0  34723.0  0.227658   49175.0
         1   0.273988  0.503796    6993.0  17517.0  0.229808   24510.0
1  0  0  0   0.271659  0.500555   28775.0  70248.0  0.228896   99023.0
         1   0.269196  0.497491   14313.0  35068.0  0.228295   49381.0
      1 

# Simulation 4 : 2 covariables avec termes croisés pour Y et Z

In [542]:
Nobs = 1000000 # nombre d'observations
alpha_tau = 1 #effet du traitement
alpha_eZ = [2,2,0,0] #effet des covariables sur le propensity score
NX = np.size(alpha_eZ) # nombre de covariables
alpha_eY = [-2,-2,0,0] #effet des covariables sur la variable d'intérêt
alphaXCrossX_eZ = np.zeros((NX,NX)) #effet croisé des covariables sur le propensity score
alphaXCrossX_eY = np.zeros((NX,NX)) #effet croisé des covariables sur la variable d'intérêt
alphaXCrossX_eZ[0,1] = -2 #effet croisé des covariables sur le propensity score
alphaXCrossX_eY[0,1] = -2 #effet croisé des covariables sur la variable d'intérêt
alphaXCrossZ_eY = [2,0,0,0]
df, tau_pf, tau_causal = f.genData(Nobs,alpha_tau,alpha_eZ,alpha_eY,alphaXCrossX_eZ,alphaXCrossX_eY,alphaXCrossZ_eY)
print('VALEURS TAU SIMULATION :')
print('tau_pf = E[Y,Z=1] - E[Y,Z=0] =', round(tau_pf,ndigits=3))   
print('tau_causal = E[Y(1)] - E[Y(0)] =', round(tau_causal ,ndigits=3))
print('')

f.print_moy_strat(df)
print('')

f.print_res_sim(df)

VALEURS TAU SIMULATION :
tau_pf = E[Y,Z=1] - E[Y,Z=0] = 0.072
tau_causal = E[Y(1)] - E[Y(0)] = 0.257

RESULTATS DE LA MOYENNE STRATIFIÉE :
             E[Y|Z=0]  E[Y|Z=1]    N(Z=0)   N(Z=1)      Diff     Poids
X1 X2 X3 X4                                                           
0  0  0  0   0.773022  0.903457  147975.0  48735.0  0.130435  196710.0
         1   0.773703  0.899513   74875.0  24650.0  0.125810   99525.0
      1  0   0.770498  0.901221   74082.0  24327.0  0.130723   98409.0
         1   0.772389  0.902652   37195.0  12368.0  0.130263   49563.0
   1  0  0   0.313570  0.554449   28571.0  69873.0  0.240879   98444.0
         1   0.316479  0.556258   14576.0  34946.0  0.239779   49522.0
      1  0   0.317532  0.553883   14619.0  34872.0  0.236351   49491.0
         1   0.311868  0.551183    7314.0  17330.0  0.239315   24644.0
1  0  0  0   0.313566  0.900951   28836.0  70218.0  0.587385   99054.0
         1   0.316217  0.902562   14392.0  35325.0  0.586345   49717.0
      1  

# Simulation 5 : 2 covariables, mais le modèle Y ne voit que la première

In [520]:
Nobs = 1000000 # nombre d'observations
alpha_tau = 1 #effet du traitement
alpha_eZ = [2,2,0,0] #effet des covariables sur le propensity score
NX = np.size(alpha_eZ) # nombre de covariables
alpha_eY = [-2,-2,0,0] #effet des covariables sur la variable d'intérêt
alphaXCrossX_eZ = np.zeros((NX,NX)) #effet croisé des covariables sur le propensity score
alphaXCrossX_eY = np.zeros((NX,NX)) #effet croisé des covariables sur la variable d'intérêt
alphaXCrossZ_eY = [0,0,0,0]
df, tau_pf, tau_causal = f.genData(Nobs,alpha_tau,alpha_eZ,alpha_eY,alphaXCrossX_eZ,alphaXCrossX_eY,alphaXCrossZ_eY)
print('VALEURS TAU SIMULATION :')
print('tau_pf = E[Y,Z=1] - E[Y,Z=0] =', round(tau_pf,ndigits=3))   
print('tau_causal = E[Y(1)] - E[Y(0)] =', round(tau_causal ,ndigits=3))
print('')

f.print_moy_strat(df)
print('')

vars_to_includeE = None
vars_to_includeY = ['X1']
f.print_res_sim(df,vars_to_includeY,vars_to_includeE)

VALEURS TAU SIMULATION :
tau_pf = E[Y,Z=1] - E[Y,Z=0] = -0.095
tau_causal = E[Y(1)] - E[Y(0)] = 0.177

RESULTATS DE LA MOYENNE STRATIFIÉE :
             E[Y|Z=0]  E[Y|Z=1]    N(Z=0)   N(Z=1)      Diff     Poids
X1 X2 X3 X4                                                           
0  0  0  0   0.731822  0.878919  157213.0  40890.0  0.147097  198103.0
         1   0.732106  0.878279   78490.0  20621.0  0.146173   99111.0
      1  0   0.731221  0.877924   78161.0  20520.0  0.146703   98681.0
         1   0.731229  0.874499   38996.0  10223.0  0.143270   49219.0
   1  0  0   0.266335  0.495471   33180.0  64698.0  0.229136   97878.0
         1   0.268115  0.498586   16713.0  32885.0  0.230471   49598.0
      1  0   0.266338  0.499358   16832.0  32712.0  0.233020   49544.0
         1   0.269845  0.496926    8390.0  16268.0  0.227081   24658.0
1  0  0  0   0.268171  0.498504   33680.0  65189.0  0.230333   98869.0
         1   0.264133  0.501704   16628.0  32868.0  0.237571   49496.0
      1 

# Simulation 5 : 2 covariables, mais le modèle Z ne voit que la première

In [524]:
Nobs = 1000000 # nombre d'observations
alpha_tau = 1 #effet du traitement
alpha_eZ = [2,2,0,0] #effet des covariables sur le propensity score
NX = np.size(alpha_eZ) # nombre de covariables
alpha_eY = [-2,-2,0,0] #effet des covariables sur la variable d'intérêt
alphaXCrossX_eZ = np.zeros((NX,NX)) #effet croisé des covariables sur le propensity score
alphaXCrossX_eY = np.zeros((NX,NX)) #effet croisé des covariables sur la variable d'intérêt
alphaXCrossZ_eY = [0,0,0,0]
df, tau_pf, tau_causal = f.genData(Nobs,alpha_tau,alpha_eZ,alpha_eY,alphaXCrossX_eZ,alphaXCrossX_eY,alphaXCrossZ_eY)
print('VALEURS TAU SIMULATION :')
print('tau_pf = E[Y,Z=1] - E[Y,Z=0] =', round(tau_pf,ndigits=3))   
print('tau_causal = E[Y(1)] - E[Y(0)] =', round(tau_causal ,ndigits=3))
print('')

f.print_moy_strat(df)
print('')

vars_to_includeE = ['X1']
vars_to_includeY = None
f.print_res_sim(df,vars_to_includeY,vars_to_includeE)

VALEURS TAU SIMULATION :
tau_pf = E[Y,Z=1] - E[Y,Z=0] = -0.094
tau_causal = E[Y(1)] - E[Y(0)] = 0.177

RESULTATS DE LA MOYENNE STRATIFIÉE :
             E[Y|Z=0]  E[Y|Z=1]    N(Z=0)   N(Z=1)      Diff     Poids
X1 X2 X3 X4                                                           
0  0  0  0   0.729654  0.881830  156322.0  40975.0  0.152176  197297.0
         1   0.730313  0.879172   77868.0  20583.0  0.148859   98451.0
      1  0   0.731653  0.880684   78298.0  20584.0  0.149031   98882.0
         1   0.732122  0.885618   38790.0  10430.0  0.153497   49220.0
   1  0  0   0.268146  0.500852   33478.0  65137.0  0.232706   98615.0
         1   0.278142  0.501081   16772.0  32382.0  0.222939   49154.0
      1  0   0.265487  0.496117   16901.0  32966.0  0.230630   49867.0
         1   0.273361  0.502313    8326.0  16215.0  0.228952   24541.0
1  0  0  0   0.271882  0.499017   33566.0  65641.0  0.227135   99207.0
         1   0.265817  0.498576   16786.0  32647.0  0.232759   49433.0
      1 

# Simulation 6

In [545]:
Nobs = 1000000 # nombre d'observations
alpha_tau = 1 #effet du traitement
alpha_eZ = [2,2,0,0] #effet des covariables sur le propensity score
NX = np.size(alpha_eZ) # nombre de covariables
alpha_eY = [-2,-2,2,2] #effet des covariables sur la variable d'intérêt
alphaXCrossX_eZ = np.zeros((NX,NX)) #effet croisé des covariables sur le propensity score
alphaXCrossX_eY = np.zeros((NX,NX)) #effet croisé des covariables sur la variable d'intérêt
alphaXCrossX_eZ[0,1] = -2 #effet croisé des covariables sur le propensity score
alphaXCrossX_eY[0,1] = -2 #effet croisé des covariables sur la variable d'intérêt
alphaXCrossZ_eY = [1,1,1,1]
df, tau_pf, tau_causal = f.genData(Nobs,alpha_tau,alpha_eZ,alpha_eY,alphaXCrossX_eZ,alphaXCrossX_eY,alphaXCrossZ_eY)
print('VALEURS TAU SIMULATION :')
print('tau_pf = E[Y,Z=1] - E[Y,Z=0] =', round(tau_pf,ndigits=3))   
print('tau_causal = E[Y(1)] - E[Y(0)] =', round(tau_causal ,ndigits=3))
print('')

f.print_moy_strat(df)
print('')

vars_to_includeE = ['X1','X2']
vars_to_includeY = ['X1','X2']
f.print_res_sim(df,vars_to_includeY,vars_to_includeE)

VALEURS TAU SIMULATION :
tau_pf = E[Y,Z=1] - E[Y,Z=0] = 0.135
tau_causal = E[Y(1)] - E[Y(0)] = 0.277

RESULTATS DE LA MOYENNE STRATIFIÉE :
             E[Y|Z=0]  E[Y|Z=1]    N(Z=0)   N(Z=1)      Diff     Poids
X1 X2 X3 X4                                                           
0  0  0  0   0.471802  0.707236  148948.0  49019.0  0.235434  197967.0
         1   0.868021  0.978584   74110.0  24421.0  0.110563   98531.0
      1  0   0.868261  0.979773   74215.0  24620.0  0.111511   98835.0
         1   0.981843  0.999347   37230.0  12245.0  0.017504   49475.0
   1  0  0   0.107023  0.469576   28807.0  70323.0  0.362554   99130.0
         1   0.473319  0.947870   14280.0  34759.0  0.474550   49039.0
      1  0   0.464495  0.945481   14336.0  35272.0  0.480986   49608.0
         1   0.872276  0.997530    7250.0  17412.0  0.125255   24662.0
1  0  0  0   0.107493  0.469229   28839.0  70132.0  0.361736   98971.0
         1   0.473629  0.946776   14334.0  34928.0  0.473147   49262.0
      1  